# Import Data

## Import what we need

In [4]:
import pandas as pd
from tqdm import tqdm

## Import Data

In [5]:
# Constants
data_dir = 'datasets/'

In [6]:
# Load datasets we are interested in
admissions_df = pd.read_csv(data_dir + 'ADMISSIONS.csv')

In [7]:
noteevents_df = pd.read_csv(data_dir + 'NOTEEVENTS.csv')

/var/folders/tr/hvbln4bd0vbfnb0dkggctvcr0000gp/T/ipykernel_36113/1774928600.py:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  noteevents_df = pd.read_csv(data_dir + 'NOTEEVENTS.csv')


# Feature Engineering

## Admissions Feature Engineering

In [8]:
admissions_df.describe

<bound method NDFrame.describe of        ROW_ID  SUBJECT_ID  HADM_ID            ADMITTIME            DISCHTIME  \
0          21          22   165315  2196-04-09 12:26:00  2196-04-10 15:54:00   
1          22          23   152223  2153-09-03 07:15:00  2153-09-08 19:10:00   
2          23          23   124321  2157-10-18 19:34:00  2157-10-25 14:00:00   
3          24          24   161859  2139-06-06 16:14:00  2139-06-09 12:48:00   
4          25          25   129635  2160-11-02 02:06:00  2160-11-05 14:55:00   
...       ...         ...      ...                  ...                  ...   
58971   58594       98800   191113  2131-03-30 21:13:00  2131-04-02 15:02:00   
58972   58595       98802   101071  2151-03-05 20:00:00  2151-03-06 09:10:00   
58973   58596       98805   122631  2200-09-12 07:15:00  2200-09-20 12:08:00   
58974   58597       98813   170407  2128-11-11 02:29:00  2128-12-22 13:11:00   
58975   58598       98813   190264  2131-10-25 03:09:00  2131-10-26 17:44:00   

     

In [9]:
admissions_df = admissions_df[["SUBJECT_ID", "HADM_ID", "DISCHTIME"]]

In [10]:
admissions_df

,SUBJECT_ID,HADM_ID,DISCHTIME
0,22,165315,2196-04-10 15:54:00
1,23,152223,2153-09-08 19:10:00
2,23,124321,2157-10-25 14:00:00
3,24,161859,2139-06-09 12:48:00
4,25,129635,2160-11-05 14:55:00
...,...,...,...
58971,98800,191113,2131-04-02 15:02:00
58972,98802,101071,2151-03-06 09:10:00
58973,98805,122631,2200-09-20 12:08:00
58974,98813,170407,2128-12-22 13:11:00


In [11]:
# Validate that all records have a unique HADM_ID
print(len(admissions_df["HADM_ID"].unique()) == len(admissions_df))

True


In [12]:
print(admissions_df['DISCHTIME'][0] > admissions_df['DISCHTIME'][1])

True


In [13]:
# Function to determine if admission is the last admission for the patient
# 1 indicates the record is the last admission for the patient while 0 indicates it was not the last admission

for patient in tqdm(admissions_df['SUBJECT_ID']):
    last_discharge = None
    for admission in admissions_df[admissions_df.SUBJECT_ID == patient].HADM_ID.values:
        current_discharge = admissions_df[(admissions_df.SUBJECT_ID == patient) & (admissions_df.HADM_ID == admission)].DISCHTIME.values[0]
        if last_discharge == None:
            last_discharge = current_discharge
            admissions_df.loc[(admissions_df.SUBJECT_ID == patient) & (admissions_df.HADM_ID == admission), 'LAST_ADMISSION'] = 1
        elif last_discharge > current_discharge:
            admissions_df.loc[(admissions_df.SUBJECT_ID == patient) & (admissions_df.HADM_ID == admission), 'LAST_ADMISSION'] = 0
        else:
            admissions_df.loc[(admissions_df.SUBJECT_ID == patient) & (admissions_df.HADM_ID == admission) & (admissions_df.DISCHTIME == last_discharge), 'LAST_ADMISSION'] = 0
            admissions_df.loc[(admissions_df.SUBJECT_ID == patient) & (admissions_df.HADM_ID == admission) & (admissions_df.DISCHTIME == current_discharge), 'LAST_ADMISSION'] = 1
            last_discharge = current_discharge

  0%|          | 0/58976 [00:00<?, ?it/s]/var/folders/tr/hvbln4bd0vbfnb0dkggctvcr0000gp/T/ipykernel_36113/2252296837.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  admissions_df.loc[(admissions_df.SUBJECT_ID == patient) & (admissions_df.HADM_ID == admission), 'LAST_ADMISSION'] = 1
100%|██████████| 58976/58976 [06:36<00:00, 148.74it/s] 


In [14]:
admissions_df

,SUBJECT_ID,HADM_ID,DISCHTIME,LAST_ADMISSION
0,22,165315,2196-04-10 15:54:00,1.0
1,23,152223,2153-09-08 19:10:00,1.0
2,23,124321,2157-10-25 14:00:00,1.0
3,24,161859,2139-06-09 12:48:00,1.0
4,25,129635,2160-11-05 14:55:00,1.0
...,...,...,...,...
58971,98800,191113,2131-04-02 15:02:00,1.0
58972,98802,101071,2151-03-06 09:10:00,1.0
58973,98805,122631,2200-09-20 12:08:00,1.0
58974,98813,170407,2128-12-22 13:11:00,0.0


In [114]:
admissions_df.drop(columns=['DISCHTIME'])

,SUBJECT_ID,HADM_ID,LAST_ADMISSION
0,22,165315,1.0
1,23,152223,1.0
2,23,124321,1.0
3,24,161859,1.0
4,25,129635,1.0
...,...,...,...
58971,98800,191113,1.0
58972,98802,101071,1.0
58973,98805,122631,1.0
58974,98813,170407,0.0


## NoteEvents Feature Engineering

In [15]:
noteevents_df

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...
...,...,...,...,...,...,...,...,...,...,...,...
2083175,2070657,31097,115637.0,2132-01-21,2132-01-21 03:27:00,2132-01-21 03:38:00,Nursing/other,Report,17581.0,NaN,NPN\n\n\n#1 Infant remains in RA with O2 sats...
2083176,2070658,31097,115637.0,2132-01-21,2132-01-21 09:50:00,2132-01-21 09:53:00,Nursing/other,Report,19211.0,NaN,"Neonatology\nDOL #5, CGA 36 weeks.\n\nCVR: Con..."
2083177,2070659,31097,115637.0,2132-01-21,2132-01-21 16:42:00,2132-01-21 16:44:00,Nursing/other,Report,20104.0,NaN,Family Meeting Note\nFamily meeting held with ...
2083178,2070660,31097,115637.0,2132-01-21,2132-01-21 18:05:00,2132-01-21 18:16:00,Nursing/other,Report,16023.0,NaN,NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...


In [16]:
non_discharge_rows = noteevents_df.drop(noteevents_df.loc[noteevents_df['CATEGORY'] == 'Discharge summary'].index)

In [17]:
non_discharge_rows['CATEGORY'].unique()

array(['Echo', 'ECG', 'Nursing', 'Physician ', 'Rehab Services',
       'Case Management ', 'Respiratory ', 'Nutrition', 'General',
       'Social Work', 'Pharmacy', 'Consult', 'Radiology', 'Nursing/other'],
      dtype=object)

In [18]:

grouped_non_discharge_rows_df = non_discharge_rows.groupby(['HADM_ID'], as_index=False)['TEXT'].agg(list)

In [19]:
print(len(noteevents_df[noteevents_df.CATEGORY == 'Discharge summary']))

len(noteevents_df[(noteevents_df.HADM_ID == 115637.0) & (noteevents_df.CATEGORY == 'Discharge summary')])

59652


1

In [20]:
# Notes other than discharge summary for each visit

for admission in tqdm(noteevents_df.HADM_ID.values):
    if admission in grouped_non_discharge_rows_df.HADM_ID.values:
        noteevents_df.loc[(noteevents_df.HADM_ID == admission), 'NOT_DS_NOTES_COUNT'] = len(grouped_non_discharge_rows_df[grouped_non_discharge_rows_df.HADM_ID == admission].TEXT.values[0])
    else:
        noteevents_df.loc[(noteevents_df.HADM_ID == admission), 'NOT_DS_NOTES_COUNT'] = 0


100%|██████████| 2083180/2083180 [39:08<00:00, 886.98it/s] 


In [21]:
noteevents_df

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,NOT_DS_NOTES_COUNT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...,30.0
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...,46.0
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...,106.0
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...,25.0
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2083175,2070657,31097,115637.0,2132-01-21,2132-01-21 03:27:00,2132-01-21 03:38:00,Nursing/other,Report,17581.0,NaN,NPN\n\n\n#1 Infant remains in RA with O2 sats...,31.0
2083176,2070658,31097,115637.0,2132-01-21,2132-01-21 09:50:00,2132-01-21 09:53:00,Nursing/other,Report,19211.0,NaN,"Neonatology\nDOL #5, CGA 36 weeks.\n\nCVR: Con...",31.0
2083177,2070659,31097,115637.0,2132-01-21,2132-01-21 16:42:00,2132-01-21 16:44:00,Nursing/other,Report,20104.0,NaN,Family Meeting Note\nFamily meeting held with ...,31.0
2083178,2070660,31097,115637.0,2132-01-21,2132-01-21 18:05:00,2132-01-21 18:16:00,Nursing/other,Report,16023.0,NaN,NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...,31.0


In [22]:
noteevents_df[noteevents_df.HADM_ID == 115637.0]

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,NOT_DS_NOTES_COUNT
34114,32296,31097,115637.0,2132-01-22,NaN,NaN,Discharge summary,Report,NaN,NaN,Unit No: [**Numeric Identifier 75487**]\nAdmi...,31.0
2081105,2070662,31097,115637.0,2132-01-21,2132-01-21 21:18:00,2132-01-21 21:20:00,Nursing/other,Report,20104.0,NaN,NNP On-Call\nPlease see Dr.[**Name (NI) 169**]...,31.0
2081106,2070663,31097,115637.0,2132-01-21,2132-01-21 22:29:00,2132-01-21 22:33:00,Nursing/other,Report,15002.0,NaN,Nursing Progress Note 1900-2300\n\n\n1. Resp:\...,31.0
2081107,2070664,31097,115637.0,2132-01-22,2132-01-22 06:07:00,2132-01-22 06:14:00,Nursing/other,Report,15226.0,NaN,NPN 11p-7a\n\n\n#1 In RA with sats 96-100. BBS...,31.0
2081108,2070665,31097,115637.0,2132-01-22,2132-01-22 09:20:00,2132-01-22 09:24:00,Nursing/other,Report,19211.0,NaN,"Neonatology\nDOL #6, CGA 36 1/7 weeks.\n\nCVR:...",31.0
2081109,2070666,31097,115637.0,2132-01-22,2132-01-22 10:37:00,2132-01-22 10:41:00,Nursing/other,Report,18706.0,NaN,Neonatology NNP note\nPE:\n\nNEURO: infant swa...,31.0
2081257,2070667,31097,115637.0,2132-01-22,2132-01-22 13:46:00,2132-01-22 13:56:00,Nursing/other,Report,19371.0,NaN,Nursing Transfer Note\n\n\n#1 RESP: Pt. remai...,31.0
2082626,2070637,31097,115637.0,2132-01-17,2132-01-17 00:45:00,2132-01-17 00:54:00,Nursing/other,Report,16888.0,NaN,Neonatology\npatient is 1860 gram product of 3...,31.0
2082627,2070638,31097,115637.0,2132-01-17,2132-01-17 01:15:00,2132-01-17 01:23:00,Nursing/other,Report,14828.0,NaN,Nursing Admission Note\nBaby boy [**Known last...,31.0
2082628,2070639,31097,115637.0,2132-01-17,2132-01-17 03:29:00,2132-01-17 03:29:00,Nursing/other,Report,14828.0,NaN,1 RESP:\n2 FEN:\n3 DEV:\n4 PAR:\n\nREVISIONS T...,31.0


In [23]:
discharge_with_count_df = noteevents_df.drop(noteevents_df.loc[noteevents_df['CATEGORY'] != 'Discharge summary'].index)

In [24]:
discharge_with_count_df

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,NOT_DS_NOTES_COUNT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...,30.0
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...,46.0
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...,106.0
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...,25.0
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...
60411,59608,93924,135672.0,2104-06-14,NaN,NaN,Discharge summary,Addendum,NaN,NaN,"Name: [**Known lastname 18321**],[**Known fir...",28.0
60412,59609,17782,155710.0,2112-03-31,NaN,NaN,Discharge summary,Addendum,NaN,NaN,"Name: [**Known lastname 2706**], [**Known fir...",9.0
60413,59621,96333,183951.0,2197-06-18,NaN,NaN,Discharge summary,Addendum,NaN,NaN,Name: [**Known lastname 18357**]-[**Known las...,10.0
60414,59622,66717,169165.0,2129-08-14,NaN,NaN,Discharge summary,Addendum,NaN,NaN,"Name: [**Known lastname **],[**Known firstnam...",17.0


In [25]:
discharge_with_count_df['TEXT'][0]

'Admission Date:  [**2151-7-16**]       Discharge Date:  [**2151-8-4**]\n\n\nService:\nADDENDUM:\n\nRADIOLOGIC STUDIES:  Radiologic studies also included a chest\nCT, which confirmed cavitary lesions in the left lung apex\nconsistent with infectious process/tuberculosis.  This also\nmoderate-sized left pleural effusion.\n\nHEAD CT:  Head CT showed no intracranial hemorrhage or mass\neffect, but old infarction consistent with past medical\nhistory.\n\nABDOMINAL CT:  Abdominal CT showed lesions of\nT10 and sacrum most likely secondary to osteoporosis. These can\nbe followed by repeat imaging as an outpatient.\n\n\n\n                            [**First Name8 (NamePattern2) **] [**First Name4 (NamePattern1) 1775**] [**Last Name (NamePattern1) **], M.D.  [**MD Number(1) 1776**]\n\nDictated By:[**Hospital 1807**]\nMEDQUIST36\n\nD:  [**2151-8-5**]  12:11\nT:  [**2151-8-5**]  12:21\nJOB#:  [**Job Number 1808**]\n'

In [26]:
discharge_with_count_df = discharge_with_count_df[['HADM_ID', 'TEXT', 'NOT_DS_NOTES_COUNT']]

In [27]:
discharge_with_count_df

,HADM_ID,TEXT,NOT_DS_NOTES_COUNT
0,167853.0,Admission Date: [**2151-7-16**] Dischar...,30.0
1,107527.0,Admission Date: [**2118-6-2**] Discharg...,46.0
2,167118.0,Admission Date: [**2119-5-4**] D...,106.0
3,196489.0,Admission Date: [**2124-7-21**] ...,25.0
4,135453.0,Admission Date: [**2162-3-3**] D...,38.0
...,...,...,...
60411,135672.0,"Name: [**Known lastname 18321**],[**Known fir...",28.0
60412,155710.0,"Name: [**Known lastname 2706**], [**Known fir...",9.0
60413,183951.0,Name: [**Known lastname 18357**]-[**Known las...,10.0
60414,169165.0,"Name: [**Known lastname **],[**Known firstnam...",17.0


In [192]:
admissions_df

,SUBJECT_ID,HADM_ID,DISCHTIME,LAST_ADMISSION
0,22,165315,2196-04-10 15:54:00,1.0
1,23,152223,2153-09-08 19:10:00,1.0
2,23,124321,2157-10-25 14:00:00,1.0
3,24,161859,2139-06-09 12:48:00,1.0
4,25,129635,2160-11-05 14:55:00,1.0
...,...,...,...,...
58971,98800,191113,2131-04-02 15:02:00,1.0
58972,98802,101071,2151-03-06 09:10:00,1.0
58973,98805,122631,2200-09-20 12:08:00,1.0
58974,98813,170407,2128-12-22 13:11:00,0.0


## Combine the two datasets

In [61]:
together_df = pd.merge(discharge_with_count_df, admissions_df, how='inner', on='HADM_ID')

In [62]:
together_df = together_df[["HADM_ID", "TEXT", "NOT_DS_NOTES_COUNT", "LAST_ADMISSION"]]

In [63]:
together_df

,HADM_ID,TEXT,NOT_DS_NOTES_COUNT,LAST_ADMISSION
0,167853.0,Admission Date: [**2151-7-16**] Dischar...,30.0,1.0
1,167853.0,Admission Date: [**2151-7-16**] Dischar...,30.0,1.0
2,107527.0,Admission Date: [**2118-6-2**] Discharg...,46.0,1.0
3,167118.0,Admission Date: [**2119-5-4**] D...,106.0,1.0
4,196489.0,Admission Date: [**2124-7-21**] ...,25.0,1.0
...,...,...,...,...
59647,129192.0,"Name: [**Known lastname 15324**], [**Known fi...",11.0,1.0
59648,117457.0,"Name: [**Known lastname 14938**], [**Known fi...",6.0,1.0
59649,171593.0,"Name: [**Known lastname **], [**Known firstna...",15.0,1.0
59650,131944.0,"Name: [**Known lastname **], [**Known firstna...",3.0,1.0


In [64]:
together_df[together_df.HADM_ID == 167853.0].values[0]

array([167853.0,
       'Admission Date:  [**2151-7-16**]       Discharge Date:  [**2151-8-4**]\n\n\nService:\nADDENDUM:\n\nRADIOLOGIC STUDIES:  Radiologic studies also included a chest\nCT, which confirmed cavitary lesions in the left lung apex\nconsistent with infectious process/tuberculosis.  This also\nmoderate-sized left pleural effusion.\n\nHEAD CT:  Head CT showed no intracranial hemorrhage or mass\neffect, but old infarction consistent with past medical\nhistory.\n\nABDOMINAL CT:  Abdominal CT showed lesions of\nT10 and sacrum most likely secondary to osteoporosis. These can\nbe followed by repeat imaging as an outpatient.\n\n\n\n                            [**First Name8 (NamePattern2) **] [**First Name4 (NamePattern1) 1775**] [**Last Name (NamePattern1) **], M.D.  [**MD Number(1) 1776**]\n\nDictated By:[**Hospital 1807**]\nMEDQUIST36\n\nD:  [**2151-8-5**]  12:11\nT:  [**2151-8-5**]  12:21\nJOB#:  [**Job Number 1808**]\n',
       30.0, 1.0], dtype=object)

In [65]:
together_df[together_df.HADM_ID == 167853.0].values[1]

array([167853.0,
       "Admission Date:  [**2151-7-16**]       Discharge Date:  [**2151-8-4**]\n\n\n\nHISTORY OF PRESENT ILLNESS:  The patient is an 86 year old\nAfrican American female who on the morning of [**7-16**], was\nfound on the floor of her unair-conditioned home by a\nrelative during the heat wave.  She was conscious but\nThe family reports she had not been drinking much and had not\nbeen feeling well one day prior to admission.  The family\nalso reports a productive cough, 30 lb weight loss in the\nlast three months, shortness of breath over the last few\nmonths, worsened by exertion and increasing edema.\n\nThe patient was taken to the Emergency Department and rectal\nand irregular.  Blood pressure was 137/76, respiratory rate 27.\nCooling measures were started in the Emergency Department.\nShe was placed on a 100% oxygen nonrebreather mask.  The\npatient gradually became alert and oriented times two.  In\nthe Emergency Department she subsequently became hypotensive\nwith

In [66]:
together_df = together_df.drop_duplicates(subset='HADM_ID', keep="last")

In [67]:
together_df

,HADM_ID,TEXT,NOT_DS_NOTES_COUNT,LAST_ADMISSION
1,167853.0,Admission Date: [**2151-7-16**] Dischar...,30.0,1.0
2,107527.0,Admission Date: [**2118-6-2**] Discharg...,46.0,1.0
3,167118.0,Admission Date: [**2119-5-4**] D...,106.0,1.0
4,196489.0,Admission Date: [**2124-7-21**] ...,25.0,1.0
5,135453.0,Admission Date: [**2162-3-3**] D...,38.0,1.0
...,...,...,...,...
59647,129192.0,"Name: [**Known lastname 15324**], [**Known fi...",11.0,1.0
59648,117457.0,"Name: [**Known lastname 14938**], [**Known fi...",6.0,1.0
59649,171593.0,"Name: [**Known lastname **], [**Known firstna...",15.0,1.0
59650,131944.0,"Name: [**Known lastname **], [**Known firstna...",3.0,1.0


In [68]:
together_df = together_df[['TEXT', 'NOT_DS_NOTES_COUNT', 'LAST_ADMISSION']]

In [69]:
together_df

,TEXT,NOT_DS_NOTES_COUNT,LAST_ADMISSION
1,Admission Date: [**2151-7-16**] Dischar...,30.0,1.0
2,Admission Date: [**2118-6-2**] Discharg...,46.0,1.0
3,Admission Date: [**2119-5-4**] D...,106.0,1.0
4,Admission Date: [**2124-7-21**] ...,25.0,1.0
5,Admission Date: [**2162-3-3**] D...,38.0,1.0
...,...,...,...
59647,"Name: [**Known lastname 15324**], [**Known fi...",11.0,1.0
59648,"Name: [**Known lastname 14938**], [**Known fi...",6.0,1.0
59649,"Name: [**Known lastname **], [**Known firstna...",15.0,1.0
59650,"Name: [**Known lastname **], [**Known firstna...",3.0,1.0


## Data Cleanup

In [70]:
together_df["TEXT"] = together_df["TEXT"].str.replace(r'\n', ' ', regex=True)

/var/folders/tr/hvbln4bd0vbfnb0dkggctvcr0000gp/T/ipykernel_36113/4279160816.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  together_df["TEXT"] = together_df["TEXT"].str.replace(r'\n', ' ', regex=True)


In [71]:
together_df['TEXT'].values[0]

"Admission Date:  [**2151-7-16**]       Discharge Date:  [**2151-8-4**]    HISTORY OF PRESENT ILLNESS:  The patient is an 86 year old African American female who on the morning of [**7-16**], was found on the floor of her unair-conditioned home by a relative during the heat wave.  She was conscious but The family reports she had not been drinking much and had not been feeling well one day prior to admission.  The family also reports a productive cough, 30 lb weight loss in the last three months, shortness of breath over the last few months, worsened by exertion and increasing edema.  The patient was taken to the Emergency Department and rectal and irregular.  Blood pressure was 137/76, respiratory rate 27. Cooling measures were started in the Emergency Department. She was placed on a 100% oxygen nonrebreather mask.  The patient gradually became alert and oriented times two.  In the Emergency Department she subsequently became hypotensive with blood pressure of 80/49, heart rate 87 and 

In [75]:
together_df["NOT_DS_NOTES_COUNT"] = together_df["NOT_DS_NOTES_COUNT"].astype(int)
together_df["LAST_ADMISSION"] = together_df["LAST_ADMISSION"].astype(int)

/var/folders/tr/hvbln4bd0vbfnb0dkggctvcr0000gp/T/ipykernel_36113/1194876426.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  together_df["NOT_DS_NOTES_COUNT"] = together_df["NOT_DS_NOTES_COUNT"].astype(int)
/var/folders/tr/hvbln4bd0vbfnb0dkggctvcr0000gp/T/ipykernel_36113/1194876426.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  together_df["LAST_ADMISSION"] = together_df["LAST_ADMISSION"].astype(int)


In [76]:
together_df

,TEXT,NOT_DS_NOTES_COUNT,LAST_ADMISSION
1,Admission Date: [**2151-7-16**] Dischar...,30,1
2,Admission Date: [**2118-6-2**] Discharg...,46,1
3,Admission Date: [**2119-5-4**] D...,106,1
4,Admission Date: [**2124-7-21**] ...,25,1
5,Admission Date: [**2162-3-3**] D...,38,1
...,...,...,...
59647,"Name: [**Known lastname 15324**], [**Known fi...",11,1
59648,"Name: [**Known lastname 14938**], [**Known fi...",6,1
59649,"Name: [**Known lastname **], [**Known firstna...",15,1
59650,"Name: [**Known lastname **], [**Known firstna...",3,1


# GEN AI PORTION

In [77]:
import torch  
from transformers import AutoTokenizer, LlamaForCausalLM
import sentencepiece
import tqdm
import sacremoses
#from mlx_lm import load, generate, convert

In [2]:
model, tokenizer = load("/Users/AxiosMD/PreTrained_Models/medguanaco-65b-GPTQ")

ERROR:root:No safetensors found in /Users/AxiosMD/PreTrained_Models/medguanaco-65b-GPTQ


FileNotFoundError: No safetensors found in /Users/AxiosMD/PreTrained_Models/medguanaco-65b-GPTQ

In [4]:
convert("/Users/AxiosMD/PreTrained_Models/medguanaco-65b-GPTQ", quantize=True)

ERROR:root:No safetensors found in /Users/AxiosMD/PreTrained_Models/medguanaco-65b-GPTQ


[INFO] Loading


FileNotFoundError: No safetensors found in /Users/AxiosMD/PreTrained_Models/medguanaco-65b-GPTQ

In [78]:
tokenizer = AutoTokenizer.from_pretrained("/Users/AxiosMD/PreTrained_Models/medguanaco-65b-GPTQ")

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [79]:
# Some llama or alpaca model 65b
base_model = "/Users/AxiosMD/PreTrained_Models/medguanaco-65b-GPTQ"
model = LlamaForCausalLM.from_pretrained(base_model, device_map='auto', load_in_8bit=True, torch_dtype=torch.float16, offload_folder = "./offload")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [4]:

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=30)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

NameError: name 'model' is not defined

In [9]:
access_token = ""

In [11]:
model = LlamaForCausalLM.from_pretrained("nmitchko/medguanaco-65b-GPTQ", token=access_token)
tokenizer = AutoTokenizer.from_pretrained("nmitchko/medguanaco-65b-GPTQ")

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=30)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

Trying to resume download...


KeyboardInterrupt: 

In [83]:
from transformers import BioGptTokenizer, BioGptForCausalLM
tokenizer = BioGptTokenizer.from_pretrained("/Users/AxiosMD/PreTrained_Models/BioGPT-Large-PubMedQA")
model = BioGptForCausalLM.from_pretrained("/Users/AxiosMD/PreTrained_Models/BioGPT-Large-PubMedQA")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

AttributeError: 'BioGptForCausalLM' object has no attribute 'encode'

In [92]:
import torch
from fairseq.models.transformer_lm import TransformerLanguageModel
m = TransformerLanguageModel.from_pretrained(
        "/Users/AxiosMD/PreTrained_Models/BioGPT-Large-PubMedQA", 
        "checkpoint.pt", 
        "data",
        tokenizer='moses', 
        bpe='fastbpe', 
        bpe_codes="data/bpecodes",
        min_len=100,
        max_len_b=1024)
m.cuda()
src_tokens = m.encode("COVID-19 is")
generate = m.generate([src_tokens], beam=5)[0]
output = m.decode(generate[0]["tokens"])
print(output)

2024-04-29 23:19:07 | INFO | fairseq.file_utils | loading archive file /Users/AxiosMD/PreTrained_Models/BioGPT-Large-PubMedQA


OSError: Model file not found: /Users/AxiosMD/PreTrained_Models/BioGPT-Large-PubMedQA/model.pt